In [39]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [216]:
# code to scrap the data from the URL and create a dataframe with combined neighbourhoods

scraped_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
html_content = BeautifulSoup(scraped_data.text, features='html.parser')
table = html_content.find("table", class_="wikitable sortable")
table_df = pd.DataFrame(columns = ['postcode', 'borough', 'neighbourhood'])
rows = table.findAll('tr')

for row in range(1, len(rows)):
    table_df = table_df.append({'postcode': rows[row].findAll('td')[0].text, 'borough':rows[row].findAll('td')[1].text, 'neighbourhood':rows[row].findAll('td')[2].text}, ignore_index=True)

table_df = table_df[table_df['borough'] != 'Not assigned']

table_df = table_df.groupby('postcode', as_index=False).apply(lambda x: pd.Series({
                                                              'postcode':x['postcode'].unique()[0],
                                                              'borough':x['borough'].unique()[0],
                                                              'neighbourhood':', '.join(x['neighbourhood'])
}))
table_df['neighbourhood'] = table_df['neighbourhood'].str.replace("\n", "")
print(table_df)

    postcode           borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T       Scarborough   
14       M1V       Scarborough   
15       M1W       Scarborough   
16       M1X       Scarborough   
17       M2H        North York   
18       M2J        North York   
19       M2K        North York   
20       M2L        North York   
21       M2M        North York   
22       M2N        North York   
23       M2P        North York   
24       M2R        North York   
25       M3A        North York   
26       M3B        North York   
27       M3C        North York   
28       M3H  

In [213]:
# udpate the neighbourhood
table_df.loc[table_df['neighbourhood'].str.contains('Not assigned'), 'neighbourhood'] = table_df.loc[table_df['neighbourhood'].str.contains('Not assigned')]['borough']
print(table_df.loc[table_df['borough'] == "Queen's Park"])

   postcode       borough neighbourhood
85      M7A  Queen's Park  Queen's Park


In [214]:
# using shape method, printing number of rows
print(table_df.shape)

(103, 3)


In [224]:
# adding coordinates columns in the processed dataframe

coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
table_df['Latitude'] = coordinates['Latitude']
table_df['Longitude'] = coordinates['Longitude']
print(table_df)

    Postal Code           Borough  \
0           M1B       Scarborough   
1           M1C       Scarborough   
2           M1E       Scarborough   
3           M1G       Scarborough   
4           M1H       Scarborough   
5           M1J       Scarborough   
6           M1K       Scarborough   
7           M1L       Scarborough   
8           M1M       Scarborough   
9           M1N       Scarborough   
10          M1P       Scarborough   
11          M1R       Scarborough   
12          M1S       Scarborough   
13          M1T       Scarborough   
14          M1V       Scarborough   
15          M1W       Scarborough   
16          M1X       Scarborough   
17          M2H        North York   
18          M2J        North York   
19          M2K        North York   
20          M2L        North York   
21          M2M        North York   
22          M2N        North York   
23          M2P        North York   
24          M2R        North York   
25          M3A        North York   
2